In [ ]:
class PayloadBuilder:
  def __init__(self):

    self.__pid = 1811
    self.__uid = 999999
    self.__dsid = 1261
    self.__dstype = 'DS'
    self.__map_ids = ['7cbe6546-89a8-4378-b7d6-16f4bcd93d36', 'dcea0891-75fa-4cbd-b40f-72986a16abf6']
    self.__sql_ids = ['1811-0', '1811-1']
    self.__params = {
    "@MultiCandType":{
        "value":["DT"], # 일자로 검색
        "type":"STRING",
        "defaultValue":""
    },
    "@MultiCandStDt":{
        "value":[], # 시작일
        "type":"STRING",
        "defaultValue":""
    },
    "@MultiCandEdDt":{
        "value":[], # 종료일
        "type":"STRING",
        "defaultValue":""
    },
    "@GenderCd":{
        "value":[], # 성별 코드
        "type":"STRING",
        "defaultValue":"[All]",
        "whereClause":"F.GENDER_CD"
    },
    "@SidoCd":{
        "value":["11","26","27","28","29","30","31","36","41","43","44","46","47","48","50","52","51","21","22","23","24","25","42","45"], # 전체 시/도 코드로 조회
        "type":"STRING",
        "defaultValue":"[All]",
        "whereClause":"C.SIDO_CD"
    },
    "@CggCd":{
        "value":["_EMPTY_VALUE_"], # 시/군/구 코드(필요 없음)
        "type":"STRING",
        "defaultValue":"[All]",
        "whereClause":"D.CGG_CD"
    },
    "@UmdCd":{
        "value":["_EMPTY_VALUE_"], # 읍/면/동 코드(필요 없음)
        "type":"STRING",
        "defaultValue":"[All]",
        "whereClause":"E.UMD_CD"
    }
  }

  def build(self, start_date, end_date, gender, version=0):

    self.__params['@MultiCandStDt']['value'].clear()
    self.__params['@MultiCandEdDt']['value'].clear()
    self.__params['@GenderCd']['value'].clear()

    stdt = "{:04d}{:02d}{:02d}".format(start_date.year, start_date.month, start_date.day)
    endt = "{:04d}{:02d}{:02d}".format(end_date.year, end_date.month, end_date.day)
    gender_code = '1' if gender == 'male' else '2'

    self.__params['@MultiCandStDt']['value'].append(stdt)
    self.__params['@MultiCandEdDt']['value'].append(endt)
    self.__params['@GenderCd']['value'].append(gender_code)

    payload = {
      "pid": self.__pid,
      "uid": self.__uid,
      "dsid": self.__dsid,
      "dstype": self.__dstype,
      "mapid": self.__map_ids[version],
      "sqlid": self.__sql_ids[version],
      "params": json.dumps(self.__params)
    }
    return payload


In [ ]:
import requests
import json
import time
from datetime import datetime, timedelta


In [ ]:
url = 'https://efamily.scourt.go.kr/ds/report/query.do'
headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
}

# 연도 조정
date = datetime(2024, 1, 1)
limit = datetime(2024, 8, 31)
pb = PayloadBuilder()

male_names = set()
female_names = set()

session = requests.Session()

while date < limit:
  start_date = datetime(date.year, date.month, date.day)
  end_date = start_date + timedelta(days=2)
  if end_date > limit:
    end_date = limit
  date = end_date + timedelta(days=1)

  args = [
      {'start_date': start_date, 'end_date': end_date, 'gender': 'male', 'version': 0},
      {'start_date': start_date, 'end_date': end_date, 'gender': 'female', 'version': 0},
  ]


  request_start = time.time()
  for arg in args:
    payload = pb.build(
        start_date=arg['start_date'],
        end_date=arg['end_date'],
        gender=arg['gender'],
        version=arg['version']
        )
    response = session.post(url, headers=headers, data=payload)
    try:
      data = response.json()['data']
      names = []

      for row in data:
        names.append(row['이름'])

      if arg['gender'] == 'male':
        male_names.update(names)
      else:
        female_names.update(names)
    except:
      print(response.text)

  request_end = time.time()

  s = "{:04d}-{:02d}-{:02d}".format(start_date.year, start_date.month, start_date.day)
  e = "{:04d}-{:02d}-{:02d}".format(end_date.year, end_date.month, end_date.day)
  print(f'{s} ~ {e}, male_names: {len(male_names)}, female_names: {len(female_names)} execution time: {request_end - request_start}')


  time.sleep(0.3)

session.close()

2024-01-01 ~ 2024-01-03, male_names: 21, female_names: 21 execution time: 2.547522783279419
2024-01-04 ~ 2024-01-06, male_names: 25, female_names: 28 execution time: 0.9069490432739258
2024-01-07 ~ 2024-01-09, male_names: 29, female_names: 36 execution time: 0.9027624130249023
2024-01-10 ~ 2024-01-12, male_names: 32, female_names: 40 execution time: 0.9334800243377686
2024-01-13 ~ 2024-01-15, male_names: 35, female_names: 41 execution time: 0.8887858390808105
2024-01-16 ~ 2024-01-18, male_names: 36, female_names: 41 execution time: 1.1975007057189941
2024-01-19 ~ 2024-01-21, male_names: 40, female_names: 45 execution time: 4.2577598094940186
2024-01-22 ~ 2024-01-24, male_names: 40, female_names: 45 execution time: 0.9844658374786377
2024-01-25 ~ 2024-01-27, male_names: 41, female_names: 47 execution time: 0.881202220916748
2024-01-28 ~ 2024-01-30, male_names: 43, female_names: 49 execution time: 0.8853926658630371
2024-01-31 ~ 2024-02-02, male_names: 43, female_names: 49 execution time

In [ ]:
print(f'male_names: {len(male_names)}, names: {list(male_names)[:min(20, len(male_names))]}')
print(f'female_names: {len(female_names)}, names: {list(female_names)[:min(20, len(female_names))]}')

male_names: 85, names: ['이안', '로운', '태이', '예준', '지우', '이찬', '준우', '선우', '재이', '주원', '도훈', '기타', '다온', '현우', '시우', '시원', '아준', '진우', '윤우', '서우']
female_names: 107, names: ['하율', '민채', '채아', '지우', '재이', '채원', '기타', '하윤', '다온', '다은', '윤슬', '나은', '수아', '에일리세아', '겨울', '해인', '예린', '서은', '유나', '이서']


In [ ]:
import csv

with open('../male_names_2024.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  for row in male_names:
    writer.writerow([row])

with open('../female_names_2024.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  for row in female_names:
    writer.writerow([row])
